In [ ]:
from google.colab import drive
import pandas as pd 
import torch
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import numpy as np
from PIL import Image
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

%matplotlib inline

In [ ]:
drive.mount('/content/gdrive')  # link google drive

In [ ]:
label_header = ["photo", "type"]
labels = pd.read_csv("/content/gdrive/My Drive/VRDL/HW1/training_labels.txt",
                     names=label_header, sep=' ')
train_fold = "/content/gdrive/MyDrive/VRDL/HW1/train_images_new"
valid_fold = "/content/gdrive/MyDrive/VRDL/HW1/validation_images"

class_header = ["type"]
class_data = pd.read_csv("/content/gdrive/My Drive/VRDL/HW1/classes.txt",
                         names=class_header)
test_fold = "/content/gdrive/MyDrive/VRDL/HW1/test_images"


In [ ]:
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

# adjust to bigger,random get 350*350,
# do other change to get more data,then normalize
train_transforms = T.Compose([
    T.Resize((375, 375)),
    T.RandomCrop((350, 350)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ToTensor(),
    T.Normalize(*imagenet_stats, inplace=True),
    T.RandomErasing(inplace=True)
])

valid_transforms = T.Compose([
    T.Resize((350, 350)),
    T.ToTensor(),
    T.Normalize(*imagenet_stats)
])


In [ ]:
# get train/valid dataset
train_dataset = ImageFolder(train_fold, train_transforms)
valid_dataset = ImageFolder(valid_fold, valid_transforms)
# get model
model = EfficientNet.from_pretrained('efficientnet-b6')


In [ ]:
# to see model's fc layer
model


In [ ]:
model.fc = torch.nn.Linear(2304, 200)

# use gpu
model = model.cuda()
try:
    model.load_state_dict(torch.load(
        '/content/gdrive/MyDrive/VRDL/HW1/weight_eff.pth'))
    model.eval()
    print('have previous weight')
except:
    print('Cannot load previous weight')

loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), 1e-4)
scheduler = lr_scheduler.ReduceLROnPlateau(
            opt, mode='max', factor=0.1, patience=1,
            verbose=True, threshold=0.0001, threshold_mode='rel',
            cooldown=0, min_lr=0, eps=1e-08)


In [ ]:
torch.cuda.empty_cache()
# Create train/valid loaders
batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size,
                              shuffle=True, num_workers=2, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size,
                              shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
valid_loss_min = np.Inf
n = 15

for epoch in range(n):
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    print('running epoch: {}'.format(epoch))

    # train the model
    model.train()
    train_acc = 0.0
    valid_acc = 0.0
    for data, label in train_dataloader:
        torch.cuda.empty_cache()
        # move tensors to GPU if CUDA is available
        data = data.cuda()
        label = label.cuda()
        # clear the gradients of all optimized variables
        opt.zero_grad()
        # forward pass: compute predicted outputs
        # by passing inputs to the model
        pred = model(data)
        pred_label = torch.argmax(pred, dim=1)

        # calculate the batch loss
        loss = loss_fn(pred, label)
        # backward pass: compute gradient of the loss
        # with respect to model parameters
        loss.backward()
        # update training loss
        train_loss += loss.item()*data.size(0)
        train_acc += np.sum(pred_label.cpu().numpy() == label.cpu().numpy())
        # change learning rate to loss
        scheduler.step(train_acc)
        # perform a single optimization step (parameter update)
        opt.step()

    # validate the model
    model.eval()
    for data, label in valid_dataloader:
        data = data.cuda()
        label = label.cuda()
        opt.zero_grad()
        pred = model(data)
        pred_label = torch.argmax(pred, dim=1)
        loss = loss_fn(pred, label)
        loss.backward()
        valid_loss += loss.item()*data.size(0)
        valid_acc += np.sum(pred_label.cpu().numpy() == label.cpu().numpy())
        opt.step()

    # calculate average losses
    train_loss = train_loss/len(train_dataloader.dataset)
    valid_loss = valid_loss/len(valid_dataloader.dataset)
    train_acc = train_acc/len(train_dataloader.dataset)
    valid_acc = valid_acc/len(valid_dataloader.dataset)

    # print training/validation statistics
    print(f'\tTraining Loss: {train_loss:.6f}')
    print(f'\tValidation Loss: {valid_loss:.6f}')
    print(f'\t Training acc: {train_acc:.6f}')
    print(f'\tValidation acc: {valid_acc:.6f}')

    if valid_loss <= valid_loss_min:
        torch.save(model.state_dict(),
                   '/content/gdrive/MyDrive/VRDL/HW1/weight_eff.pth')
        print('Validation loss decreased ({:.6f} --> {:.6f}).Saving model ...'
              .format(valid_loss_min, valid_loss))
